In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_data(csv_file): 
    return pd.read_csv(csv_file,
                       encoding = 'utf-8') 
data_form = load_data(r"K_origin.csv") 
data_form.head()

In [ ]:
from mp_api.client import MPRester
from pymatgen.io.cif import CifWriter

API_KEY = ""
mpr = MPRester(API_KEY)

for i in range(len(data_form)):
    id = data_form.iloc[i, 0]
    structure = mpr.get_structure_by_material_id(id)
    CifWriter(structure, 0.1).write_file("./K_cifs/"+id+".cif")

In [ ]:
import os
import re
from pymatgen.core import Structure


cif_dir = r'./K_cifs'
cif_files= os.listdir(cif_dir)

c=[]
for f in cif_files:
    if f.endswith(".cif"):
        
        f = re.sub("\D", "", f)
        c.append(f)
c_1 = sorted(list(map(int, c)))
new_cif_files = [(str(i)+'.cif') for i in c_1]
cif_paths = [os.path.join(cif_dir, cif) for cif in new_cif_files]


structures = [Structure.from_file(cif) for cif in cif_paths]
structure_df = pd.DataFrame({"structure": structures})

In [ ]:
structure_df.head()

In [ ]:
len(data_form)

In [ ]:
# Import matminer featureizers
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.function import FunctionFeaturizer
from matminer.featurizers.conversions import CompositionToOxidComposition
from pymatgen.core import Structure, Composition
from matminer.featurizers.conversions import StrToComposition, StructureToComposition

# Composition featureizers
from matminer.featurizers.composition import (
    ElementProperty, Stoichiometry, ValenceOrbital, IonProperty, 
    ElementFraction, Meredig, YangSolidSolution, CohesiveEnergy, 
    ElectronegativityDiff, ElectronAffinity, AtomicOrbitals, OxidationStates
)

# Structure featureizers
from matminer.featurizers.structure import (
    DensityFeatures, 
    RadialDistributionFunction, 
    GlobalSymmetryFeatures,
    BondFractions, 
    ChemicalOrdering, 
    MaximumPackingEfficiency,
    StructuralHeterogeneity, 
    SineCoulombMatrix, 
    CoulombMatrix,
    EwaldEnergy, 
    BagofBonds, 
    OrbitalFieldMatrix,
    PartialRadialDistributionFunction, 
    XRDPowderPattern,
    SiteStatsFingerprint
)

# Site featureizers
from matminer.featurizers.site import (
    CrystalNNFingerprint, 
    AGNIFingerprints,
    VoronoiFingerprint,
    OPSiteFingerprint, 
    ChemEnvSiteFingerprint, 
    GaussianSymmFunc, 
)

### composition

In [ ]:
structure_df['formula'] = [structure.composition.formula for structure in structure_df['structure']]
structure_df['composition'] = [structure.composition for structure in structure_df['structure']]
structure_df.head()

In [ ]:
# 1.1 Basic element properties
try:
    element_property = ElementProperty.from_preset('magpie')
    element_features = element_property.featurize_dataframe(structure_df.copy(), col_id='composition')
    print(f"ElementProperty feature extraction complete, total {len(element_features.columns)-3} features")
except Exception as e:
    print(f"ElementProperty feature extraction failed: {e}")

# 1.2 Stoichiometry features
try:
    stoichiometry = Stoichiometry()
    stoichiometry_features = stoichiometry.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"Stoichiometry feature extraction complete, total {len(stoichiometry_features.columns)-3} features")
except Exception as e:
    print(f"Stoichiometry feature extraction failed: {e}")

# 1.3 Valence orbital features
try:
    valence_orbital = ValenceOrbital()
    valence_features = valence_orbital.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"ValenceOrbital feature extraction complete, total {len(valence_features.columns)-3} features")
except Exception as e:
    print(f"ValenceOrbital feature extraction failed: {e}")

# 1.4 Ion properties
try:
    ion_property = IonProperty()
    ion_features = ion_property.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"IonProperty feature extraction complete, total {len(ion_features.columns)-3} features")
except Exception as e:
    print(f"IonProperty feature extraction failed: {e}")

# 1.5 Element fraction
try:
    element_fraction = ElementFraction()
    fraction_features = element_fraction.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"ElementFraction feature extraction complete, total {len(fraction_features.columns)-3} features")
except Exception as e:
    print(f"ElementFraction feature extraction failed: {e}")

# 1.6 Meredig features
try:
    meredig = Meredig()
    meredig_features = meredig.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"Meredig feature extraction complete, total {len(meredig_features.columns)-3} features")
except Exception as e:
    print(f"Meredig feature extraction failed: {e}")

# 1.7 Yang solid solution features
try:
    yang_solid_solution = YangSolidSolution()
    yang_features = yang_solid_solution.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"YangSolidSolution feature extraction complete, total {len(yang_features.columns)-3} features")
except Exception as e:
    print(f"YangSolidSolution feature extraction failed: {e}")

# 1.8 Cohesive energy
try:
    cohesive_energy = CohesiveEnergy()
    cohesive_features = cohesive_energy.featurize_dataframe(structure_df.copy(), col_id='composition', ignore_errors=True)
    print(f"CohesiveEnergy feature extraction complete, total {len(cohesive_features.columns)-3} features")
except Exception as e:
    print(f"CohesiveEnergy feature extraction failed: {e}")




In [ ]:
dfs1 = [element_features, stoichiometry_features, valence_features, ion_features, fraction_features, \
                    meredig_features, yang_features, cohesive_features]

In [ ]:
merged_df1 = pd.DataFrame()
for df in dfs1:
    new_gaussian_cols = [col for col in df.columns if col not in structure_df.columns]
    merged_df1 = pd.concat([merged_df1, df[new_gaussian_cols]], axis=1)

In [ ]:
merged_df1.head()

In [ ]:
# 1.9  Add oxidation states
addcomp_addmag_add = CompositionToOxidComposition().featurize_dataframe(structure_df, "composition",ignore_errors=True)
addcomp_addmag_add.head()
## Oxidation states
os_feat = OxidationStates()
oxid_features = os_feat.featurize_dataframe(addcomp_addmag_add, "composition_oxid",ignore_errors=True)
oxid_features.head()

In [ ]:
structure_df = oxid_features[["structure", "composition", "composition_oxid"]]
structure_df.head()

In [ ]:
# 1.10 Electronegativity difference
try:
    electronegativity_diff = ElectronegativityDiff()
    electronegativity_features = electronegativity_diff.featurize_dataframe(structure_df.copy(), col_id='composition_oxid', ignore_errors=True)
    print(f"ElectronegativityDiff feature extraction complete, total {len(electronegativity_features.columns)-3} features")
except Exception as e:
    print(f"ElectronegativityDiff feature extraction failed: {e}")

# 1.11 Electron affinity
try:
    electron_affinity = ElectronAffinity()
    electron_affinity_features = electron_affinity.featurize_dataframe(structure_df.copy(), col_id='composition_oxid', ignore_errors=True)
    print(f"ElectronAffinity feature extraction complete, total {len(electron_affinity_features.columns)-3} features")
except Exception as e:
    print(f"ElectronAffinity feature extraction failed: {e}")

# 1.12 Atomic orbital features
try:
    atomic_orbitals = AtomicOrbitals()
    atomic_orbitals_features = atomic_orbitals.featurize_dataframe(structure_df.copy(), col_id='composition_oxid', ignore_errors=True)
    print(f"AtomicOrbitals feature extraction complete, total {len(atomic_orbitals_features.columns)-3} features")
except Exception as e:
    print(f"AtomicOrbitals feature extraction failed: {e}")

In [ ]:
dfs2 = [oxid_features, electronegativity_features, electron_affinity_features, atomic_orbitals_features]

In [ ]:
merged_df2 = pd.DataFrame()
for df in dfs2:
    new_gaussian_cols = [col for col in df.columns if col not in structure_df.columns]
    merged_df2 = pd.concat([merged_df2, df[new_gaussian_cols]], axis=1)

In [ ]:
merged_df2.head()

In [ ]:
df1 = pd.concat([merged_df1, merged_df2], axis=1)
numeric_df1 = df1.select_dtypes(include=['number', 'bool'])
# Calculate NaN ratio for each column
nan_ratio = numeric_df1.isnull().sum() / len(numeric_df1)

# Find columns with NaN ratio less than or equal to threshold
columns_to_keep = nan_ratio[nan_ratio <= 0.01].index

# Keep these columns
numeric_df1 = numeric_df1[columns_to_keep]
numeric_df1.head()

#### structure

In [ ]:
# 2. Structure feature extraction
print("\nStarting structure feature extraction...")

# 2.1 Density features
try:
    density_features = DensityFeatures(desired_features=['vpa', 'packing fraction'])
    density_features_df = density_features.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"DensityFeatures feature extraction complete, total {len(density_features_df.columns)-3} features")
except Exception as e:
    print(f"DensityFeatures feature extraction failed: {e}")

# 2.2 Radial distribution function
try:
    rdf = RadialDistributionFunction()
    rdf_features = rdf.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"RadialDistributionFunction feature extraction complete, total {len(rdf_features.columns)-3} features")
except Exception as e:
    print(f"RadialDistributionFunction feature extraction failed: {e}")

# 2.3 Global symmetry features
try:
    global_symmetry = GlobalSymmetryFeatures()
    symmetry_features = global_symmetry.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"GlobalSymmetryFeatures feature extraction complete, total {len(symmetry_features.columns)-3} features")
except Exception as e:
    print(f"GlobalSymmetryFeatures feature extraction failed: {e}")

# 2.4 Bond fractions
try:
    bond_fractions = BondFractions()
    bond_fractions.fit(structure_df['structure'])
    bond_features = bond_fractions.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"BondFractions feature extraction complete, total {len(bond_features.columns)-3} features")
except Exception as e:
    print(f"BondFractions feature extraction failed: {e}")

# 2.5 Chemical ordering
try:
    chemical_ordering = ChemicalOrdering()
    chemical_ordering_features = chemical_ordering.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"ChemicalOrdering feature extraction complete, total {len(chemical_ordering_features.columns)-3} features")
except Exception as e:
    print(f"ChemicalOrdering feature extraction failed: {e}")



In [ ]:
# 2.6 Maximum packing efficiency
try:
    max_packing = MaximumPackingEfficiency()
    max_packing_features = max_packing.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"MaximumPackingEfficiency feature extraction complete, total {len(max_packing_features.columns)-3} features")
except Exception as e:
    print(f"MaximumPackingEfficiency feature extraction failed: {e}")

# 2.7 Structural heterogeneity
try:
    structural_heterogeneity = StructuralHeterogeneity()
    heterogeneity_features = structural_heterogeneity.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"StructuralHeterogeneity feature extraction complete, total {len(heterogeneity_features.columns)-3} features")
except Exception as e:
    print(f"StructuralHeterogeneity feature extraction failed: {e}")

# 2.8 Matrix featurizers
try:
    # Sine Coulomb matrix
    sine_coulomb = SineCoulombMatrix()
    sine_coulomb.fit(structure_df['structure'])
    sine_coulomb_features = sine_coulomb.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"SineCoulombMatrix feature extraction complete, total {len(sine_coulomb_features.columns)-3} features")
    
    # Coulomb matrix
    coulomb_matrix = CoulombMatrix()
    coulomb_matrix.fit(structure_df['structure'])
    coulomb_matrix_features = coulomb_matrix.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"CoulombMatrix feature extraction complete, total {len(coulomb_matrix_features.columns)-3} features")
    
    # Ewald energy
    ewald_energy = EwaldEnergy()
    ewald_energy_features = ewald_energy.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"EwaldEnergy feature extraction complete, total {len(ewald_energy_features.columns)-3} features")
except Exception as e:
    print(f"Matrix featurizer extraction failed: {e}")

# 2.9 XRD powder diffraction features
try:
    # Create XRDPowderPattern featurizer
    xrd = XRDPowderPattern(two_theta_range=(0, 127), bw_method=0.05)
    
    # Get feature labels
    feature_labels = xrd.feature_labels()
    
    # Record column count before featurization
    initial_columns = len(structure_df.columns)
    
    # Apply featurization
    xrd_features = xrd.featurize_dataframe(structure_df.copy(), col_id='structure')
    print(f"XRD feature label count: {len(xrd_features.columns)-3}")
except Exception as e:
    print(f"XRDPowderPattern feature extraction failed: {e}")

# 2.10 Other structure featurizers
try:
    # Orbital field matrix
    orbital_field_matrix = OrbitalFieldMatrix()
    orbital_field_features = orbital_field_matrix.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"OrbitalFieldMatrix feature extraction complete, total {len(orbital_field_features.columns)-3} features")

except Exception as e:
    print(f"OrbitalFieldMatrix featurizer extraction failed: {e}")



In [ ]:
dfs3 = [density_features_df, rdf_features, symmetry_features, bond_features, chemical_ordering_features,\
                        max_packing_features, heterogeneity_features, sine_coulomb_features, coulomb_matrix_features, ewald_energy_features,\
                                    xrd_features, orbital_field_features]
merged_df3 = pd.DataFrame()

In [ ]:
for df in dfs3:
    new_gaussian_cols = [col for col in df.columns if col not in structure_df.columns]
    merged_df3 = pd.concat([merged_df3, df[new_gaussian_cols]], axis=1)
merged_df3.head()

In [ ]:

numeric_df2 = merged_df3.select_dtypes(include=['number', 'bool'])
# Calculate NaN ratio for each column
nan_ratio = numeric_df2.isnull().sum() / len(numeric_df2)

# Find columns with NaN ratio less than or equal to threshold
columns_to_keep = nan_ratio[nan_ratio <= 0.0001].index

# Keep these columns
numeric_df2 = numeric_df2[columns_to_keep]
numeric_df2.head()

#### Site

In [ ]:
# 3. Site feature extraction
print("\nStarting site feature extraction...")
# 3.1 CrystalNN fingerprint
try:
    crystal_nn = CrystalNNFingerprint.from_preset("cn")
    site_stats = SiteStatsFingerprint(crystal_nn)
    crystal_nn_features = site_stats.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"CrystalNNFingerprint feature extraction complete, total {len(crystal_nn_features.columns)-3} features")
except Exception as e:
    print(f"CrystalNNFingerprint feature extraction failed: {e}")

# 3.2 AGNI fingerprint
try:
    agni = AGNIFingerprints()
    site_stats_agni = SiteStatsFingerprint(agni)
    agni_features = site_stats_agni.featurize_dataframe(structure_df.copy(), col_id='structure',ignore_errors=True)
    print(f"AGNIFingerprints feature extraction complete, total {len(agni_features.columns)-3} features")
except Exception as e:
    print(f"AGNIFingerprints feature extraction failed: {e}")

# 3.3 Voronoi fingerprint
try:
    voronoi = VoronoiFingerprint()
    site_stats_voronoi = SiteStatsFingerprint(voronoi)
    voronoi_features = site_stats_voronoi.featurize_dataframe(structure_df.copy(), col_id='structure',ignore_errors=True)
    print(f"VoronoiFingerprint feature extraction complete, total {len(voronoi_features.columns)-3} features")
except Exception as e:
    print(f"VoronoiFingerprint feature extraction failed: {e}")

# 3.4 OP site fingerprint
try:
    op_site = OPSiteFingerprint()
    site_stats_op = SiteStatsFingerprint(op_site)
    op_features = site_stats_op.featurize_dataframe(structure_df.copy(), col_id='structure', ignore_errors=True)
    print(f"OPSiteFingerprint feature extraction complete, total {len(op_features.columns)-3} features")
except Exception as e:
    print(f"OPSiteFingerprint feature extraction failed: {e}")

# 3.5 Chemical environment site fingerprint
try:
    chem_env = ChemEnvSiteFingerprint.from_preset("simple")
    site_stats_chem_env = SiteStatsFingerprint(chem_env)
    chem_env_features = site_stats_chem_env.featurize_dataframe(structure_df.copy(), col_id='structure',ignore_errors=True)
    print(f"ChemEnvSiteFingerprint feature extraction complete, total {len(chem_env_features.columns)-3} features")
except Exception as e:
    print(f"ChemEnvSiteFingerprint feature extraction failed: {e}")

# 3.6 Gaussian symmetry function
try:
    gaussian_symm = GaussianSymmFunc()
    site_stats_gaussian = SiteStatsFingerprint(gaussian_symm)
    gaussian_features = site_stats_gaussian.featurize_dataframe(structure_df.copy(), col_id='structure',ignore_errors=True)
    print(f"GaussianSymmFunc feature extraction complete, total {len(gaussian_features.columns)-3} features")
except Exception as e:
    print(f"GaussianSymmFunc feature extraction failed: {e}")


In [ ]:
dfs4 = [crystal_nn_features, agni_features, voronoi_features, op_features, chem_env_features, gaussian_features]

In [ ]:
merged_df4 = pd.DataFrame()
for df in dfs4:
    new_gaussian_cols = [col for col in df.columns if col not in structure_df.columns]
    merged_df4 = pd.concat([merged_df4, df[new_gaussian_cols]], axis=1)

In [ ]:
numeric_df3 = merged_df4.select_dtypes(include=['number', 'bool'])
# Calculate NaN ratio for each column
nan_ratio = numeric_df3.isnull().sum() / len(numeric_df3)

# Find columns with NaN ratio less than or equal to threshold
columns_to_keep = nan_ratio[nan_ratio <= 0.0001].index

# Keep these columns
numeric_df3 = numeric_df3[columns_to_keep]
numeric_df3.head()

In [ ]:
features_all = pd.concat([numeric_df1, numeric_df2, numeric_df3], axis=1)
features_all.head()

In [ ]:
features_all.insert(0, 'material_id', data_set['material_id'])

In [ ]:
for column in data_set.columns:
    if column in data_set.columns and column not in features_all.columns:
        features_all[column] = data_set[column]

In [ ]:
features_all.head()